In [8]:
import os
from pathlib import Path
import pandas as pd
import pyranges as pr
import peft
from peft import PeftType
from functools import partial

from greyhound.model import GreyhoundConfig, Greyhound
from greyhound.model.locon import add_locon, merge_locon
from greyhound.data import ChromatinDataset, train_filter, val_filter, test_filter
from enformer_pytorch import GenomeIntervalDataset
from transformers import Trainer, TrainingArguments

In [2]:
model_config = GreyhoundConfig(
    borzoi_model_name="johahi/borzoi-replicate-0",
    n_labels=2,
    use_autocast=True,
    borzoi_kwargs={
        "enable_mouse_head": False,
    },
)
model = Greyhound(config=model_config)
model.init_borzoi_weights()

Loaded Borzoi weights from johahi/borzoi-replicate-0 into Greyhound model.


In [3]:
model

Greyhound(
  (borzoi): Borzoi(
    (conv_dna): ConvDna(
      (conv_layer): Conv1d(4, 512, kernel_size=(15,), stride=(1,), padding=same)
      (max_pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (_max_pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (res_tower): Sequential(
      (0): ConvBlock(
        (norm): BatchNorm1d(512, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (activation): GELU(approximate='tanh')
        (conv_layer): Conv1d(512, 608, kernel_size=(5,), stride=(1,), padding=same)
      )
      (1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): ConvBlock(
        (norm): BatchNorm1d(608, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (activation): GELU(approximate='tanh')
        (conv_layer): Conv1d(608, 736, kernel_size=(5,), stride=(1,), padding=same)
      )
      (3): MaxPool1d(kernel_size=2, strid

In [4]:
model = add_locon(model, **{"r": 8, "lora_alpha": 16}, conv_select=4)

2025-08-03 15:19:30.043 | INFO     | greyhound.model.locon:add_locon:128 - Total parameters before LoRA and LoCon: 185896541
2025-08-03 15:19:30.044 | INFO     | greyhound.model.locon:add_locon:129 - Number of parameters after LoRA: 331522
2025-08-03 15:19:30.044 | INFO     | greyhound.model.locon:add_locon:130 - Number of parameters after LoCon: 395004
2025-08-03 15:19:30.044 | INFO     | greyhound.model.locon:add_locon:131 - Total trainable parameters after LoRA and LoCon: 395004


In [12]:
merge_locon(model)

All LoRA and LoCon weights merged into the base model.


PeftModelForFeatureExtraction(
  (base_model): LoraModel(
    (model): Greyhound(
      (borzoi): Borzoi(
        (conv_dna): ConvDna(
          (conv_layer): Conv1d(4, 512, kernel_size=(15,), stride=(1,), padding=same)
          (max_pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        )
        (_max_pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        (res_tower): Sequential(
          (0): ConvBlock(
            (norm): BatchNorm1d(512, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (activation): GELU(approximate='tanh')
            (conv_layer): Conv1d(512, 608, kernel_size=(5,), stride=(1,), padding=same)
          )
          (1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
          (2): ConvBlock(
            (norm): BatchNorm1d(608, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (activation): GELU(approximate='tanh'

In [11]:
# Reload the module to get the fixed merge_locon function
import importlib
import greyhound.model.locon
importlib.reload(greyhound.model.locon)
from greyhound.model.locon import merge_locon

# Check with a bit of data

In [6]:
genome_datasets = {}
for dataset in {"train": train_filter, "val": val_filter, "test": test_filter}.items():
    name, filter_func = dataset

    filter_func = partial(
        filter_func,
        test_fold=3,  # Adjust these values as needed
        val_fold=4,  # Adjust these values as needed
    )

    genome_datasets[name] = GenomeIntervalDataset(
        bed_file="/Users/asmith/Documents/software/greyhound/data/resources/sequences_human.bed.gz",
        fasta_file="/Users/asmith/Documents/reference/hg38.fa",
        return_augs=True,
        rc_aug=True,
        return_seq_indices=False,
        shift_augs=[-3, 3],
        context_length=524_288,
        filter_df_fn=filter_func,
    )


ds_train = ChromatinDataset(
    genome_dataset=genome_datasets["train"],
    bigwig_dir="/Users/asmith/Desktop/borzoi-training-data/"
)
ds_val = ChromatinDataset(
    genome_dataset=genome_datasets["val"],
    bigwig_dir="/Users/asmith/Desktop/borzoi-training-data/"
)
ds_test = ChromatinDataset(
    genome_dataset=genome_datasets["test"],
    bigwig_dir="/Users/asmith/Desktop/borzoi-training-data/"
)


In [7]:
# training_args = TrainingArguments(
#     bf16_full_eval=False,
#     bf16=False,
#     dataloader_num_workers=4,
#     dataloader_pin_memory=True,
#     eval_accumulation_steps=10,
#     eval_steps=5,
#     eval_strategy="steps",
#     gradient_accumulation_steps=8,
#     label_names=["labels"],
#     learning_rate=1e-4,
#     load_best_model_at_end=True,
#     logging_steps=10,
#     logging_dir="logs",
#     num_train_epochs=5,
#     output_dir="checkpoints/locon",
#     per_device_eval_batch_size=1,
#     per_device_train_batch_size=2,
#     prediction_loss_only=False,
#     remove_unused_columns=False,
#     report_to="wandb",
#     save_steps=5,
#     lr_scheduler_type="cosine",
#     save_strategy="steps",
#     weight_decay=1e-6,
#     use_mps_device=True
# )
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=ds_train,
#     eval_dataset=ds_test,
# )

# ##### TRAINING #####
# trainer.train()

In [ ]:
# Test the updated add_locon function that excludes chromatin_head
import torch
import importlib
import greyhound.model.locon
importlib.reload(greyhound.model.locon)
from greyhound.model.locon import add_locon, merge_locon

# Create a fresh model to test with
model_config = GreyhoundConfig(
    borzoi_model_name="johahi/borzoi-replicate-0",
    n_labels=2,
    use_autocast=True,
    borzoi_kwargs={
        "enable_mouse_head": False,
    },
)
test_model = Greyhound(config=model_config)
test_model.init_borzoi_weights()

# Check which Conv1D layers are found (should exclude chromatin_head)
conv_layers = [(name, module) for name, module in test_model.named_modules() 
               if isinstance(module, torch.nn.Conv1d)]
print("All Conv1D layers:")
for name, _ in conv_layers:
    print(f"  {name}")

# Check which Conv1D layers will be targeted by LoCon (excluding chromatin_head)
filtered_conv_layers = [(name, module) for name, module in test_model.named_modules()
                       if isinstance(module, torch.nn.Conv1d) and "chromatin_head" not in name]
print(f"\nConv1D layers that will be targeted by LoCon (excluding chromatin_head):")
for name, _ in filtered_conv_layers:
    print(f"  {name}")

print(f"\nTotal Conv1D layers: {len(conv_layers)}")
print(f"Conv1D layers for LoCon: {len(filtered_conv_layers)}")

Loaded Borzoi weights from johahi/borzoi-replicate-0 into Greyhound model.


NameError: name 'torch' is not defined

In [ ]:
# Test the new generic ignore_layers parameter
import torch
import re
import importlib
import greyhound.model.locon
importlib.reload(greyhound.model.locon)
from greyhound.model.locon import add_locon, merge_locon

# Create a fresh model to test with
model_config = GreyhoundConfig(
    borzoi_model_name="johahi/borzoi-replicate-0",
    n_labels=2,
    use_autocast=True,
    borzoi_kwargs={
        "enable_mouse_head": False,
    },
)
test_model = Greyhound(config=model_config)
test_model.init_borzoi_weights()

# List all Conv1D layers
conv_layers = [(name, module) for name, module in test_model.named_modules() 
               if isinstance(module, torch.nn.Conv1d)]
print("All Conv1D layers:")
for name, _ in conv_layers:
    print(f"  {name}")

print("\n" + "="*50)

# Test 1: Default behavior (ignores chromatin_head)
print("Test 1: Default ignore (chromatin_head)")
test_model1 = Greyhound(config=model_config)
test_model1.init_borzoi_weights()
test_model1 = add_locon(test_model1, rank=4, alpha=1.0, conv_select=2, r=8, lora_alpha=16)

print("\n" + "="*50)

# Test 2: Ignore multiple patterns using list
print("Test 2: Ignore multiple patterns ['chromatin', 'head']")
test_model2 = Greyhound(config=model_config)
test_model2.init_borzoi_weights()
test_model2 = add_locon(test_model2, rank=4, alpha=1.0, conv_select=2, 
                       ignore_layers=['chromatin', 'head'], r=8, lora_alpha=16)

print("\n" + "="*50)

# Test 3: Ignore using regex pattern
print("Test 3: Ignore using regex pattern (anything ending with 'head')")
test_model3 = Greyhound(config=model_config)
test_model3.init_borzoi_weights()
test_model3 = add_locon(test_model3, rank=4, alpha=1.0, conv_select=2,
                       ignore_layers=[re.compile(r'.*head$')], r=8, lora_alpha=16)

print("\n" + "="*50)

# Test 4: No ignore patterns (empty list)
print("Test 4: No ignore patterns (apply to all Conv1D layers)")
test_model4 = Greyhound(config=model_config)
test_model4.init_borzoi_weights()
test_model4 = add_locon(test_model4, rank=4, alpha=1.0, conv_select=2,
                       ignore_layers=[], r=8, lora_alpha=16)